# Chapter 1
## Tensor decomposition

This section deals with a simple concept: to decompose a 2D matrix, so far, we have _vectorised_ it, i.e. we made a long thing vector of a rectangular matrix. What if we did not need to? Because we actually didn't. 

Tensor decomposition extends matrix decomposition to N-dimensional arrays, i.e. tensors. The principle is simple: if SVD decomposes a matrix $X$ into:

$$
X = \sum_{j=1}^{m} \sigma_j * U_j \circ V_j
$$

i.e. a sum of rank-1 matrices where $U_j$ and $V_j$ are a column and row vector respsectively, then we can certainly extend this to the 3 dimensional case, just by adding a outer product to a new vector (that captures the third dimension mode, e.g. time, if x and y are space)

$$
X = \sum_{j=1}^{m} \sigma_j * U_j \circ V_j \circ Z_j
$$

this is, again, the sum of rank 1 3D arrays (tensors). 

Let's see an example.

`x=-5:1:5

y=-6:1:6 

t=0:1:10*pi

[X,Y,T]=meshgrid(x,y,t); 

A=exp(-(X.^2+0.5*Y.^2)).*(cos(2*T))+ (sech(X).*tanh(X).*exp(-0.2*Y.^2)).*sin(T)`

In [14]:
# x=-5:0.1:5
# y=-6:0.1:6
# t=0:0.1:10*π

x = 1:4
y = 5:7
t = 1:2

1:2

In [46]:
X=repeat(x', length(y),1,length(t));
Y=repeat(y, 1, length(x),length(t));
# repeat(t, length(y), length(x))
T=repeat(reshape(t, 1,1,length(t)),length(y),length(x))
# test
A=exp.(-(X.^2+0.5.*Y.^2)).*(cos.(2*T)).+ (sech.(X).*tanh.(X).*exp.(-0.2.*Y.^2)).*sin.(T)

MethodError: MethodError: no method matching exp(::Array{Float64, 3})
Closest candidates are:
  exp(!Matched::Union{Float16, Float32, Float64}) at C:\Users\Filippo Menolascina\.julia\juliaup\julia-1.7.3+0.x64\share\julia\base\special\exp.jl:296
  exp(!Matched::StridedMatrix{var"#s861"} where var"#s861"<:Union{Float32, Float64, ComplexF32, ComplexF64}) at C:\Users\Filippo Menolascina\.julia\juliaup\julia-1.7.3+0.x64\share\julia\stdlib\v1.7\LinearAlgebra\src\dense.jl:560
  exp(!Matched::StridedMatrix{var"#s861"} where var"#s861"<:Union{Integer, Complex{<:Integer}}) at C:\Users\Filippo Menolascina\.julia\juliaup\julia-1.7.3+0.x64\share\julia\stdlib\v1.7\LinearAlgebra\src\dense.jl:561
  ...